In [4]:
import matplotlib as mpl
import matplotlib.pyplot as plt
#将matplotlib的图表直接嵌入到notebook，它有一个指定图表显示的参数，inline表示将图表嵌入到notebook
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time

import tensorflow as tf
from tensorflow import keras

print(tf.__version__)
for module in mpl,np,pd,sklearn,tf,keras:
    print(module.__name__, module.__version__)

2.0.0-alpha0
matplotlib 3.1.1
numpy 1.16.3
pandas 0.24.2
sklearn 0.21.2
tensorflow 2.0.0-alpha0
tensorflow.python.keras.api._v2.keras 2.2.4-tf


In [7]:
#定义常量矩阵tensor
t = tf.constant([[1.,2.,3.],[4.,5.,6.]])  #2x3的矩阵
print(t)
print(t[:,1:])
print(t[...,1])  #...取所有行
print(t[...,1:])

tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[2. 3.]
 [5. 6.]], shape=(2, 2), dtype=float32)
tf.Tensor([2. 5.], shape=(2,), dtype=float32)
tf.Tensor(
[[2. 3.]
 [5. 6.]], shape=(2, 2), dtype=float32)


In [8]:
#矩阵操作
print(t+10)
print(tf.square(t))        
print(t @ tf.transpose(t))  #@代表矩阵相乘

tf.Tensor(
[[11. 12. 13.]
 [14. 15. 16.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[ 1.  4.  9.]
 [16. 25. 36.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[14. 32.]
 [32. 77.]], shape=(2, 2), dtype=float32)


In [10]:
#numpy conversion numpy转化为矩阵
print(t.numpy())   #将tensor 转化为numpy
print(np.square(t))
np_t = np.array([[1.,2.,3.],[4.,5.,6.]])
print(tf.constant(np_t))  #将numpy 转化为tensor

[[1. 2. 3.]
 [4. 5. 6.]]
[[ 1.  4.  9.]
 [16. 25. 36.]]
tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float64)


In [14]:
#Scalars
t = tf.constant(2.718)
print(t.numpy())
print(t.shape)
#strings
t = tf.constant('cafe')
print(t)
print(tf.strings.length(t))
print(tf.strings.length(t,unit="UTF8_CHAR"))
print(tf.strings.unicode_decode(t,"UTF8"))

2.718
()
tf.Tensor(b'cafe', shape=(), dtype=string)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor([ 99  97 102 101], shape=(4,), dtype=int32)


In [15]:
#string array
t = tf.constant(["cafe","coffee","咖啡"])
print(tf.strings.length(t,unit="UTF8_CHAR"))
r = tf.strings.unicode_decode(t,"UTF8")
print(r)

tf.Tensor([4 6 2], shape=(3,), dtype=int32)
<tf.RaggedTensor [[99, 97, 102, 101], [99, 111, 102, 102, 101, 101], [21654, 21857]]>


In [17]:
# ragger tensor
#ragger tensor 主要用于处理数据格式不规范的数据，不像矩阵那样行列固定
r = tf.ragged.constant([[11,12],[21,22,23],[],[41]])
print(r)
print(r[1])
print(r[1:2])

<tf.RaggedTensor [[11, 12], [21, 22, 23], [], [41]]>
tf.Tensor([21 22 23], shape=(3,), dtype=int32)
<tf.RaggedTensor [[21, 22, 23]]>


In [18]:
#ops on ragger tensor
r2 = tf.ragged.constant([[51,52],[],[71]])
print(tf.concat([r,r2],axis =0))  # 按照行进行拼接

<tf.RaggedTensor [[11, 12], [21, 22, 23], [], [41], [51, 52], [], [71]]>


In [19]:
#按照列进行拼接的时候一定要保证行数相等
r3 = tf.ragged.constant([[13,14],[15],[],[71]])
print(tf.concat([r,r3],axis=1))

<tf.RaggedTensor [[11, 12, 13, 14], [21, 22, 23, 15], [], [41, 71]]>


In [21]:
#将ragged tensor  转化为tensor，末尾补充0
print(r.to_tensor())

tf.Tensor(
[[11 12  0]
 [21 22 23]
 [ 0  0  0]
 [41  0  0]], shape=(4, 3), dtype=int32)


In [33]:
##sparse tensor
#indices 代表系数tensor中有值的索引位置
#values 代表有值索引位置对应的具体的数值
#dense_shape 代表tensor的维度
s = tf.SparseTensor(indices=[[0,1],[1,0],[2,3]],
                    values=[1.,2.,3.],
                    dense_shape=[3,4])
print(s)
print(tf.sparse.to_dense(s))

SparseTensor(indices=tf.Tensor(
[[0 1]
 [1 0]
 [2 3]], shape=(3, 2), dtype=int64), values=tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))
tf.Tensor(
[[0. 1. 0. 0.]
 [2. 0. 0. 0.]
 [0. 0. 0. 3.]], shape=(3, 4), dtype=float32)


In [35]:
#ops on sparse tensor
s2 = s*2.0
print(s2)
#系数tensor 不能操作+
try:
    s3 = s+1
except TypeError as ex:
    print(ex)
s4 = tf.constant([[10.,20.],
                  [30.,40.],
                  [50.,60.],
                  [70.,80.]])
#系数tensor 乘法
print(tf.sparse.sparse_dense_matmul(s,s4))

SparseTensor(indices=tf.Tensor(
[[0 1]
 [1 0]
 [2 3]], shape=(3, 2), dtype=int64), values=tf.Tensor([2. 4. 6.], shape=(3,), dtype=float32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))
unsupported operand type(s) for +: 'SparseTensor' and 'int'
tf.Tensor(
[[ 30.  40.]
 [ 20.  40.]
 [210. 240.]], shape=(3, 2), dtype=float32)


In [44]:
#sparse tensor
#稀疏tensor 的索引一定是有顺序的，顺序不对则会报错，但是可以用reorder函数进行调整
s5 = tf.SparseTensor(indices = [[0, 2], [0, 1], [2, 3]],
                    values = [1., 2., 3.],
                    dense_shape = [3, 4])
print(s5)
s6 = tf.sparse.reorder(s5)
print(tf.sparse.to_dense(s6))

SparseTensor(indices=tf.Tensor(
[[0 2]
 [0 1]
 [2 3]], shape=(3, 2), dtype=int64), values=tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))
tf.Tensor(
[[0. 2. 1. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 3.]], shape=(3, 4), dtype=float32)


In [45]:
#Variables
v = tf.Variable([[1., 2., 3.], [4., 5., 6.]])
print(v)
print(v.value())
print(v.numpy())

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>
tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float32)
[[1. 2. 3.]
 [4. 5. 6.]]


In [50]:
# assign value，变量必须用此函数进行更改
v.assign(2*v)
print(v.numpy())
v[0,1].assign(2)
print(v.numpy)
v[1].assign([7.,8.,9.])
print(v.numpy())

[[ 2.  4.  6.]
 [ 8. 10. 12.]]
<bound method ResourceVariable.numpy of <tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  2.,  6.],
       [ 8., 10., 12.]], dtype=float32)>>
[[2. 2. 6.]
 [7. 8. 9.]]


In [51]:
try:
    v[1] = [7., 8., 9.]
except TypeError as ex:
    print(ex)

'ResourceVariable' object does not support item assignment
